# Import modules

In [257]:
!pip install graphene
import graphene
import json
import sys, io
import pandas as pd
import numpy as np
import importlib
import os
import pathlib
import requests
import torch

bundle_root = pathlib.Path(os.environ.get('LABS_BUNDLE_ROOT', '/labs'))
sys.path.append(str(bundle_root / 'functions/index_by_nationality'))

In [258]:
query_metadata = """query metadata {nationalityList, jobList}"""
query_data = """query data {movieList(nationality: "Russian", job: "Director")
                {
                title
                revenue
                }}"""

# Test handler functions

In [259]:
import handler 
importlib.reload(handler);

In [ ]:
handler.handle()

In [274]:
%%timeit
handler.handle(query_data)

27.8 ms ± 573 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [283]:
with open(str(bundle_root / 'functions/index_by_nationality/schema.json'), 'w') as fp:
    json.dump(handler.schema.introspect(), fp)

# Deploy function

### cheat sheet

In [6]:
# labs bundle deploy staging (deploy to staging)
# labs bundle build classify_name (build locally for debugging)
# labs function run sunburst -d '{"date_range": [20180201,20180401], "max_level":4, "threshold":1000, "operating_system": "all"}' (example local call)

In [ ]:
#labs bundle build classify_name && docker run -it --rm --name function_test -p 5000:5000 classify_name python index.py

# Test function 

In [263]:
function_url = 'https://labs.cloud.contiamo.com/21b899ae-3eba-56a7-97da-fa26c050d1c5/v1/function/26a6b514-0324-4078-abc7-953d851c3aa4/index_by_nationality'

In [277]:
%%timeit
deployed_output = requests.post(function_url,
                                data="""query metadata {nationalityList, jobList}""").json()

215 ms ± 11.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [273]:
deployed_output

{'movieList': [{'title': 'Jurassic World', 'revenue': 1513528810},
  {'title': 'The Fate of the Furious', 'revenue': 1238764765},
  {'title': 'Captain America: Civil War', 'revenue': 1153304495}]}

### Compare

In [165]:
output['sunburst_data'] == deployed_output['sunburst_data']

True